In [89]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
import json
import numpy as np

from pyspark.sql import Row
from datetime import datetime
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [90]:
sc =SparkContext()
sqlContext = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-7d46b6d2d3b8>:1 

In [91]:
reviews = sc.textFile('filter_dataset/yelp.train.rating').map(lambda l: l.split("\t"))
reviews.take(5)

[['0', '0', '3', '2015-03-19'],
 ['0', '1', '3', '2015-03-19'],
 ['0', '2', '4', '2015-03-19'],
 ['0', '3', '4', '2015-03-19'],
 ['0', '4', '3', '2015-03-19']]

In [92]:
ratings = reviews.map(lambda l: Rating(int(l[0]),int(l[1]),float(l[2])))

In [93]:
rank = 100
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

In [94]:
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))

ratesAndPreds = ratings.map(lambda l: ((l[0],l[1]),l[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error On Training Data= " + str(MSE))

Mean Squared Error On Training Data= 0.003013220754863967


In [95]:
test_reviews = sc.textFile('filter_dataset/yelp.test.rating').map(lambda l: l.split("\t")).map(lambda l: ((int(l[0]),int(l[1])),int(l[2])))
testdata = test_reviews.map(lambda p: (p[0][0], p[0][1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))

ratesAndPreds = test_reviews.join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error On Test Data = " + str(MSE))

Mean Squared Error On Test Data = 2.4955397071962864


In [104]:
from pyspark.ml.recommendation import ALS

als = ALS(userCol = "user_id", itemCol ="buss_id", ratingCol= "rating")

rank = [100, 125, 150]
maxIter = [17, 18, 19, 20]
reg =  [.15, .17, .18, .19]
param_grid = ParamGridBuilder().addGrid(als.rank, rank).addGrid(als.maxIter, maxIter).addGrid(als.regParam, reg).build()

In [105]:
evaluator = RegressionEvaluator(metricName= "rmse", labelCol= "rating", predictionCol= "prediction")

cv = TrainValidationSplit(estimator= als, estimatorParamMaps = param_grid, evaluator= evaluator)

In [106]:
#model = cv.fit(reviews.toDF("user_id", "buss_id", "rating")
review_df = sqlContext.createDataFrame(reviews.map(lambda x: Row(user_id=int(x[0]), buss_id=int(x[1]), rating=int(x[2]))))

In [107]:
model = cv.fit(review_df)

KeyboardInterrupt: 

In [ ]:
best_model = model.bestModel

In [ ]:
testdata = sqlContext.createDataFrame(testdata.map(lambda x: Row(user_id=int(x[0]), buss_id=int(x[1]))))
predictions = best_model.transform(testdata)
rmse = evaluator.evaluate(predictions)
print("RMSE with test data is = ", rmse)